In [ ]:
!pip uninstall -y sentence-transformers

In [1]:
!pip3 install --upgrade tensorflow-gpu
# Install TF-Hub.
!pip3 install tensorflow-hub

  Using cached tensorflow_gpu-2.7.0-cp37-cp37m-manylinux2010_x86_64.whl (489.6 MB)
  Using cached tensorflow_estimator-2.7.0-py2.py3-none-any.whl (463 kB)
  Using cached tensorflow_io_gcs_filesystem-0.23.1-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (2.1 MB)
  Using cached libclang-12.0.0-py2.py3-none-manylinux1_x86_64.whl (13.4 MB)
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Attempting uninstall: tensorflow-gpu
    Found existing installation: tensorflow-gpu 2.3.0
    Uninstalling tensorflow-gpu-2.3.0:
      Successfully uninstalled tensorflow-gpu-2.3.0
  Using cached tensorflow_hub-0.12.0-py2.py3-none-any.whl (108 kB)


In [1]:
import pandas as pd
import numpy as np
import networkx as nx
from tqdm import tqdm

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" 
model = hub.load(module_url)
print ("module %s loaded" % module_url)

2021-12-25 17:10:12.248645: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


ModuleNotFoundError: No module named 'tensorflow_hub'

In [3]:
global_dataset = "SSORC_CS_2010_2021" #"SSORC_Philosophy_2000_2021"

In [4]:
papers_features = pd.read_csv("processed_data/" + global_dataset + "_papers_features.csv", index_col = 0)

/home/user/conda/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [5]:
papers_edges = pd.read_csv("processed_data/" + global_dataset + "_papers_edge_list.csv", index_col = 0)

In [6]:
authors_edges = pd.read_csv("processed_data/" + global_dataset + "_authors_edge_list.csv", index_col = 0)

In [18]:
authors_edges

,from,to
0,1404113169,2064312724
1,1404113169,1419382500
2,1404113169,2365393
3,1404113169,143638168
4,1404113169,1693033
...,...,...
30796744,101883681,143937657
30796745,101883681,9904205
30796746,101883681,145414870
30796747,143937657,9904205


In [7]:
authors_papers = pd.read_csv("processed_data/" + global_dataset + "_authors_edges_papers.csv", index_col = 0)

In [20]:
papers_id

0          984774e366d3d4fcf0fce659f697478dccd4f93c
1          8f604e61182948798ddee8ed11791017271f40d8
2          8e52ecee2bbaeb80d5ab9a11849476e86997e89d
3          f0eed1fe1e047de11e6a8e01862138b26200b6ce
4          1d1b4025f4029c99d5f87f1750b71044a137e181
                             ...                   
2682704    6be5b43a54dcf7b5b5799d8e79b178ba32fa143a
2682705    5ba1672d78b8a80bba82189222b22640181e01f4
2682706    dc42b2bec435fad0b1e66172c434edc03ff1c419
2682707    faa494fc680d79110a975bb35cba72b2ed5ebd77
2682708    071c7bd3edf07fb5d80eb4b11078a172f0359651
Name: id, Length: 2682709, dtype: object

In [8]:
papers_id = papers_features["id"]
id_to_index_id = {papers_id[i]:i for i in tqdm(range(len(papers_id)))}

100%|██████████| 2682709/2682709 [00:27<00:00, 97576.30it/s] 


In [9]:
authors_papers_unzipped = authors_papers["papers_ids"].map(lambda x: x.replace("[", "").replace("]", "").replace("'", "").split(", "))

In [11]:
authors_papers_indexed = [[id_to_index_id[authors_papers_unzipped[i][j]] for j in range(len(authors_papers_unzipped[i]))] for i in tqdm(range(len(authors_papers_unzipped)))]

100%|██████████| 30796749/30796749 [14:19<00:00, 35835.00it/s] 


In [12]:
authors_papers_indexed_str = [str(authors_papers_indexed[i]) for i in tqdm(range(len(authors_papers_indexed)))]

100%|██████████| 30796749/30796749 [00:17<00:00, 1772642.16it/s]


In [17]:
authors_papers_unzipped

0                  [984774e366d3d4fcf0fce659f697478dccd4f93c]
1                  [984774e366d3d4fcf0fce659f697478dccd4f93c]
2           [984774e366d3d4fcf0fce659f697478dccd4f93c, fdd...
3           [984774e366d3d4fcf0fce659f697478dccd4f93c, fdd...
4           [984774e366d3d4fcf0fce659f697478dccd4f93c, fdd...
                                  ...                        
30796744           [071c7bd3edf07fb5d80eb4b11078a172f0359651]
30796745           [071c7bd3edf07fb5d80eb4b11078a172f0359651]
30796746           [071c7bd3edf07fb5d80eb4b11078a172f0359651]
30796747           [071c7bd3edf07fb5d80eb4b11078a172f0359651]
30796748           [071c7bd3edf07fb5d80eb4b11078a172f0359651]
Name: papers_ids, Length: 30796749, dtype: object

In [21]:
papers_id[3]

'f0eed1fe1e047de11e6a8e01862138b26200b6ce'

In [22]:
id_to_index_id['f0eed1fe1e047de11e6a8e01862138b26200b6ce']

3

In [13]:
pd.DataFrame(authors_papers_indexed_str, columns = ["papers_indices"]).to_csv("processed_data/" + global_dataset + "_authors_edges_papers_indices.csv")

In [14]:
df = papers_features[papers_features[['id', 'title', 'paperAbstract', 'year', 'journalName', 'fieldsOfStudy']].notna()]

In [12]:
papers_features_abstracts = list(papers_features["paperAbstract"])
papers_features_abstracts = [str(papers_features_abstracts[i]) for i in range(len(papers_features_abstracts))]

In [13]:
papers_features["paperAbstract"] = papers_features["paperAbstract"].fillna("No abstract provided")

In [14]:
papers_features_abstracts[2322:2350]

['Load Balancing plays a dynamic role in keeping the tempo of the Cloud Computing framework. This research paper proposes a Forest Optimization Algorithm for load balancing in distributed computing structure. This depends on the conduct of the trees in the forest and uses the seed dispersal strategies for streamlining the makespan, which results in improved average response time and the total execution time. The simulation results prove that the proposed algorithm gives better outcomes than its counterpart load balancing algorithms.',
 'nan',
 'The classical concept lattices express the precise relation between object sets and attribute sets, but fuzzy concept lattices express the uncertain relation between object sets and attribute sets. Therefore, it is important to study hierarchy fuzzy knowledge from a fuzzy formal context. In this paper, a kind of fuzzy decision formal context is proposed and (α, β) reduct based on this fuzzy decision formal context is defined. Furthermore, we pro

In [15]:
vectorized_abstracts = []
for i in tqdm(range(len(papers_features_abstracts))):
    abstract = papers_features_abstracts[i]
    vectorized_abstracts.append(model([abstract])[0])

100%|██████████| 2682709/2682709 [1:45:57<00:00, 421.95it/s]


In [16]:
vectorized_abstracts_list = [vectorized_abstracts[i].numpy() for i in tqdm(range(len(vectorized_abstracts)))]

100%|██████████| 2682709/2682709 [00:12<00:00, 218287.87it/s]


In [17]:
vectorized_abstracts_df = pd.DataFrame(vectorized_abstracts_list)

In [18]:
print(7)

7


In [21]:
vectorized_abstracts_df.to_csv("processed_data/" + global_dataset + "_papers_features_vectorized_512.csv")

In [20]:
vectorized_abstracts_df

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
0,-0.059651,-0.058564,-0.043585,-0.055376,0.040285,-0.059395,0.042767,0.054109,-0.033267,0.060300,...,-0.010115,-0.060441,-0.005488,-0.058637,0.028976,0.057778,-0.015628,0.060211,-0.057778,-0.060236
1,-0.020976,-0.016075,-0.021127,-0.066995,-0.012023,-0.006514,-0.045765,-0.060472,0.070335,0.057117,...,0.037128,-0.075268,-0.043568,-0.062431,0.059275,0.047357,0.016749,0.030577,-0.030481,-0.056620
2,-0.058635,-0.045142,-0.013223,-0.057510,-0.058827,-0.045073,0.049923,-0.037553,0.048730,0.054082,...,0.018901,-0.065250,-0.055963,-0.034436,0.033379,0.064603,-0.006494,0.065023,-0.055938,-0.057465
3,-0.012918,-0.058709,-0.057442,0.022687,-0.030490,0.002568,0.063857,-0.041285,-0.008176,0.017138,...,0.026759,-0.066561,0.032484,0.034660,0.014997,0.047447,0.049137,-0.007857,-0.045889,-0.023475
4,-0.039173,-0.034146,-0.020552,0.022493,0.060880,0.061192,-0.044590,0.065717,0.029225,0.043425,...,-0.060069,-0.067727,-0.062527,-0.032236,-0.008576,0.056625,-0.028855,-0.061394,-0.031723,-0.027810
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2682704,-0.027332,-0.053045,-0.030270,-0.007131,-0.002052,0.044308,-0.051748,0.022303,0.053135,-0.020410,...,-0.050926,-0.060742,-0.057255,-0.040409,0.050371,-0.051289,0.057758,-0.057443,0.032680,-0.051241
2682705,-0.045520,-0.060718,-0.010936,-0.034423,-0.021102,-0.001715,-0.039156,0.009681,0.046584,0.049113,...,-0.023909,-0.062251,-0.052949,-0.048105,0.039565,-0.003266,-0.000487,0.016957,-0.039132,-0.026351
2682706,-0.060206,-0.058392,-0.023982,-0.057334,-0.047456,-0.002678,-0.027173,0.033519,-0.008071,0.060020,...,-0.025248,-0.061354,-0.059058,-0.060890,0.011957,0.057697,0.060536,0.060914,-0.020374,-0.055883
2682707,-0.002958,-0.054142,-0.058888,-0.038984,-0.039324,-0.059528,-0.042302,0.048775,-0.014610,-0.007164,...,-0.038994,-0.061646,-0.047807,-0.056433,-0.060892,0.048175,0.060353,-0.003527,-0.048387,-0.027356


In [ ]:
X = vectorized_abstracts_df

In [27]:
from sklearn.decomposition import PCA

pca = PCA(n_components=32)
pca_result = pca.fit_transform(X)

In [28]:
pd.DataFrame(pca_result).to_csv("processed_data/" + global_dataset + "_papers_features_vectorized_compressed_32.csv")

In [29]:
papers_edge_list_indexed = papers_edges.values
for i in tqdm(range(len(papers_edge_list_indexed))):
    pair = papers_edge_list_indexed[i]
    for j in range(len(pair)):
        pair[j] = id_to_index_id[pair[j]]

100%|██████████| 65202/65202 [00:00<00:00, 576920.42it/s]


In [30]:
papers_edge_list_indexed_np = pd.DataFrame(papers_edge_list_indexed)

In [31]:
papers_edge_list_indexed_np.to_csv("processed_data/" + global_dataset + "_papers_edge_list_indexed.csv")

### Execute after subdataset extraction
Journals of papers.

In [25]:
dataset_name = 'SSORC_CS_10_21_1437_3164_unfiltered' #'SSORC_CS_10_21_3340_16830_115907_primus'

In [26]:
citation_graph = nx.read_edgelist("datasets/" + dataset_name + "/" + dataset_name + "_" + "papers.edgelist", create_using = nx.DiGraph)

nodes = list(citation_graph.nodes())
nodes = [int(nodes[i]) for i in range(len(nodes))]

papers_features.loc[nodes, :]["journalName"].to_frame().to_csv("datasets/" + dataset_name + "/" + dataset_name + "_journals.csv")